In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('final_fwdcitation.csv')

/var/folders/c6/b8__gkhn17s5n3tqscb4k5300000gn/T/ipykernel_55506/3616696086.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_fwdcitation.csv')


In [4]:
df

,patent_id,patent_type,patent_date,patent_title,wipo_kind,forward_citations
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,13
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,0
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,0
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,2
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,0
...,...,...,...,...,...,...
7507814,RE50254,reissue,2024-12-31,Electrode assembly including electrode plates ...,E1,0
7507815,RE50255,reissue,2024-12-31,Mobile terminal,E1,0
7507816,RE50256,reissue,2024-12-31,"Oscillation circuit, oscillator, electronic ap...",E1,0
7507817,RE50257,reissue,2024-12-31,Data processing device and data processing method,E1,0


In [5]:
cpc = pd.read_csv('g_cpc_current.tsv', sep='\t')

In [6]:
cpc

,patent_id,cpc_sequence,cpc_section,cpc_class,cpc_subclass,cpc_group,cpc_type
0,3950000,0,A,A63,A63C,A63C9/001,inventional
1,3950000,1,A,A63,A63C,A63C9/00,inventional
2,3950000,2,A,A63,A63C,A63C9/002,inventional
3,3950000,3,A,A63,A63C,A63C9/081,inventional
4,3950001,0,A,A63,A63C,A63C9/086,inventional
...,...,...,...,...,...,...,...
56521532,12249999,2,H,H03,H03M,H03M1/18,inventional
56521533,12249999,3,H,H03,H03M,H03M1/36,inventional
56521534,12249999,4,H,H03,H03M,H03M1/56,inventional
56521535,12249999,5,H,H04,H04N,H04N25/77,inventional


In [6]:
import numpy as np

In [8]:
cpc['patent_id'] = cpc['patent_id'].astype(str)
cpc_cited = cpc[['patent_id', 'cpc_group']]

# Function to calculate entropy for a chunk
def process_entropy_chunk(chunk):
    chunk['citation_patent_id'] = chunk['citation_patent_id'].astype(str)

    # Merge to get CPC codes for cited patents
    citation_cpc = chunk[['patent_id', 'citation_patent_id']].merge(
        cpc_cited, left_on='citation_patent_id', right_on='patent_id', how='inner'
    )

    # Group by citing patent and CPC group
    entropy_counts = citation_cpc.groupby(['patent_id_x', 'cpc_group']).size().reset_index(name='count')

    # Compute Entropy
    def compute_entropy(df):
        counts = df['count']
        proportions = counts / counts.sum()
        return -np.sum(proportions * np.log(proportions))

    entropy_scores = entropy_counts.groupby('patent_id_x').apply(compute_entropy).reset_index(name='entropy_score')
    entropy_scores = entropy_scores.rename(columns={'patent_id_x': 'citing_patent_id'})

    return entropy_scores

# Initialize empty list to collect results
entropy_results = []

# Read and process citation file in chunks of 10 million rows
citation_reader = pd.read_csv('g_us_patent_citation.tsv', sep='\t', chunksize=10_000_000, low_memory=False)

for idx, chunk in enumerate(citation_reader):
    print(f"Processing chunk {idx+1}...")
    chunk_result = process_entropy_chunk(chunk)
    entropy_results.append(chunk_result)  # Save results in list

# Concatenate all results at the end
final_entropy_df = pd.concat(entropy_results, ignore_index=True)

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...


In [9]:
final_entropy_df.to_csv('entropy_scores_final.csv', index=False)

In [10]:
final_entropy_df

,citing_patent_id,entropy_score
0,10000000,1.386294
1,10000001,3.090460
2,10000002,3.856571
3,10000003,2.917433
4,10000004,3.332205
...,...,...
7738356,RE50353,2.681219
7738357,RE50354,2.484907
7738358,RE50355,6.369763
7738359,RE50356,4.367717


In [13]:
# Analyzing the entropy_scores dataframe to provide summaries

nan_summary = final_entropy_df.isna().sum()
nan_proportion = (nan_summary / len(final_entropy_df)) * 100

summary_results = {
    "Total Entries": len(final_entropy_df),
    "NaN Values": int(nan_summary['entropy_score']),
    "NaN Proportion (%)": round(nan_proportion['entropy_score'], 2)
}

summary_df = pd.DataFrame([summary_results])
summary_df

,Total Entries,NaN Values,NaN Proportion (%)
0,7738361,0,0.0


In [7]:
#Measure 2 - Technical Leap Score

In [8]:
# Ensure both patent_id columns are strings for consistent comparison
#df['patent_id'] = df['patent_id'].astype(str)
#cpc['patent_id'] = cpc['patent_id'].astype(str)

# Get the intersection count
#common_patents = df['patent_id'].isin(cpc['patent_id'])
#count_common = common_patents.sum()

print(f"Number of patents in df that also occur in cpc: {count_common}")

Number of patents in df that also occur in cpc: 6744779


In [10]:
# Count unique patents
#unique_in_df = df['patent_id'].nunique()
#unique_in_cpc = cpc['patent_id'].nunique()

print(f"Unique patents in df: {unique_in_df}")
print(f"Unique patents in cpc: {unique_in_cpc}")

Unique patents in df: 7507819
Unique patents in cpc: 8263827


In [13]:
#calculations - starting now
# Given that we have ensured patent_id is string and no missing IDs, proceed:
# Set of target patents to compute leap score for
target_patent_ids = set(df['patent_id'])
print(f"Number of target patents: {len(target_patent_ids)}")

Number of target patents: 7507819


In [14]:
citation_file = "g_us_patent_citation.tsv"
chunksize = 5_000_000  # Tune as needed

citation_dict = {}

for chunk in pd.read_csv(citation_file, sep="\t", dtype=str, usecols=['patent_id', 'citation_patent_id'], chunksize=chunksize):
    chunk = chunk[chunk['patent_id'].isin(target_patent_ids)]
    for pid, group in chunk.groupby('patent_id'):
        citation_dict.setdefault(pid, set()).update(group['citation_patent_id'])

print(f"Sample citation entry:", list(citation_dict.items())[:1])
print(f"Total patents with citation data: {len(citation_dict)}")

Sample citation entry: [('10000000', {'5093563', '5751830'})]
Total patents with citation data: 6836822


In [15]:
# Ensure types are strings to build cpc_dict from CPC Data
cpc['cpc_group'] = cpc['cpc_group'].astype(str)

# Map: patent_id -> set of CPC groups
cpc_dict = cpc.groupby('patent_id')['cpc_group'].apply(set).to_dict()

print(f"Sample CPC entry:", list(cpc_dict.items())[:1])

Sample CPC entry: [('10000000', {'G01S7/4865', 'G01S7/4917', 'G01S7/4914', 'G01S17/894', 'G01S7/4863'})]


In [20]:
import pickle

# Save all relevant data to disk
with open("leap_score_data.pkl", "wb") as f:
    pickle.dump({
        "citation_dict": citation_dict,
        "cpc_dict": cpc_dict,
        "target_patent_ids": target_patent_ids,
    }, f)

print("Saved successfully.")

Saved successfully.


In [23]:
import numpy as np

# Convert target IDs to a sorted list to chunk
target_patent_ids = sorted(list(target_patent_ids))
chunk_size = 1_000_000  # Tune this value based on memory and speed
all_results = []

for i in range(0, len(target_patent_ids), chunk_size):
    chunk_patents = target_patent_ids[i:i+chunk_size]
    chunk_results = []

    for patent_id in chunk_patents:
        citing_cpcs = cpc_dict.get(patent_id, set())
        cited_ids = citation_dict.get(patent_id, set())

        cited_cpcs = set()
        for cited_id in cited_ids:
            cited_cpcs.update(cpc_dict.get(cited_id, set()))

        if citing_cpcs or cited_cpcs:
            union = citing_cpcs.union(cited_cpcs)
            intersection = citing_cpcs.intersection(cited_cpcs)
            similarity = len(intersection) / len(union) if union else np.nan
            leap_score = 1 - similarity if not np.isnan(similarity) else np.nan
        else:
            leap_score = np.nan

        chunk_results.append((patent_id, leap_score))
    
    all_results.extend(chunk_results)
    print(f"Processed chunk {i // chunk_size + 1} of {len(target_patent_ids) // chunk_size + 1}")

KeyboardInterrupt: 

In [ ]:
# Final output
leap_df = pd.DataFrame(all_results, columns=['patent_id', 'technological_leap_score'])

In [24]:
unique_groups = cpc['cpc_group'].nunique()
unique_subclasses = cpc['cpc_subclass'].nunique()

print(f"Unique CPC Groups: {unique_groups}")
print(f"Unique CPC Subclasses: {unique_subclasses}")

Unique CPC Groups: 244725
Unique CPC Subclasses: 676


In [10]:
import numpy as np

In [11]:
chunk1['citation_patent_id'] = chunk1['citation_patent_id'].astype(str)
cpc['patent_id'] = cpc['patent_id'].astype(str)

# Load only required columns from cpc data
cpc_cited = cpc[['patent_id', 'cpc_group']]

# Merge to get CPC codes for cited patents
ci  tation_cpc = chunk1[['patent_id', 'citation_patent_id']].merge(
    cpc_cited, left_on='citation_patent_id', right_on='patent_id', how='inner'
)

# Group by citing patent and cpc_group to count category occurrences
entropy_counts = citation_cpc.groupby(['patent_id_x', 'cpc_group']).size().reset_index(name='count')

# Entropy Calculation Function
def compute_entropy(df):
    counts = df['count']
    proportions = counts / counts.sum()
    return -np.sum(proportions * np.log(proportions))

# Compute entropy per citing patent
entropy_scores = entropy_counts.groupby('patent_id_x').apply(compute_entropy).reset_index(name='entropy_score')
entropy_scores = entropy_scores.rename(columns={'patent_id_x': 'citing_patent_id'})

entropy_scores.head()

,citing_patent_id,entropy_score
0,10000000,1.386294
1,10000001,3.090460
2,10000002,3.856571
3,10000003,2.917433
4,10000004,3.332205


In [12]:
# Analyzing the entropy_scores dataframe to provide summaries

nan_summary = entropy_scores.isna().sum()
nan_proportion = (nan_summary / len(entropy_scores)) * 100

summary_results = {
    "Total Entries": len(entropy_scores),
    "NaN Values": int(nan_summary['entropy_score']),
    "NaN Proportion (%)": round(nan_proportion['entropy_score'], 2)
}

summary_df = pd.DataFrame([summary_results])
summary_df

,Total Entries,NaN Values,NaN Proportion (%)
0,432802,0,0.0


In [ ]:
from itertools import combinations
from collections import Counter

# Step 1: Extract unique CPC codes per citing patent (avoid creating massive lists in memory)
cpc_groups = citation_cpc.groupby('patent_id_x')['cpc_group'].apply(set).reset_index()

# Step 2: Efficiently count CPC pairs globally using a Counter
pair_counter = Counter()

for _, row in cpc_groups.iterrows():
    cpcs = sorted(row['cpc_group'])
    if len(cpcs) > 1:
        pair_counter.update(combinations(cpcs, 2))

# Step 3: Convert Counter to DataFrame for fast lookups
pair_freq_df = pd.DataFrame(pair_counter.items(), columns=['pair', 'pair_count'])
pair_freq_df[['cpc1', 'cpc2']] = pd.DataFrame(pair_freq_df['pair'].tolist(), index=pair_freq_df.index)
pair_freq_df.drop(columns='pair', inplace=True)

In [ ]:
# Step 4: Calculate atypicality score (inverse of frequency)
pair_freq_df['atypicality_score'] = 1 / pair_freq_df['pair_count']

# Step 5: Aggregate atypicality scores per patent without materializing all pairs
atypicality_scores = []

for _, row in cpc_groups.iterrows():
    cpcs = sorted(row['cpc_group'])
    if len(cpcs) > 1:
        pairs = combinations(cpcs, 2)
        scores = [pair_freq_df.loc[(pair_freq_df['cpc1'] == p[0]) & (pair_freq_df['cpc2'] == p[1]), 'atypicality_score'].values[0] 
                  for p in pairs]
        atypicality_scores.append({'patent_id': row['patent_id_x'], 
                                   'atypicality_score': np.mean(scores)})
    else:
        atypicality_scores.append({'patent_id': row['patent_id_x'], 
                                   'atypicality_score': np.nan})

atypicality_scores_df = pd.DataFrame(atypicality_scores)

atypicality_scores_df.head()

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('')

In [2]:
#MEASURE 1: Part I
import pandas as pd
import numpy as np
df = pd.read_csv('entropy_scores_final.csv')

/var/folders/c6/b8__gkhn17s5n3tqscb4k5300000gn/T/ipykernel_33595/3782742379.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('entropy_scores_final.csv')


In [3]:
df

,citing_patent_id,entropy_score
0,10000000,1.386294
1,10000001,3.090460
2,10000002,3.856571
3,10000003,2.917433
4,10000004,3.332205
...,...,...
7738356,RE50353,2.681219
7738357,RE50354,2.484907
7738358,RE50355,6.369763
7738359,RE50356,4.367717


In [4]:
data = pd.read_csv('tech_leap_scores_final.csv')
data

/var/folders/c6/b8__gkhn17s5n3tqscb4k5300000gn/T/ipykernel_33595/3119400265.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('tech_leap_scores_final.csv')


,patent_id,tech_leap_score
0,10000000,1.000000
1,10000001,0.906250
2,10000002,0.939024
3,10000003,0.846154
4,10000004,0.980769
...,...,...
8438095,RE50353,1.000000
8438096,RE50354,1.000000
8438097,RE50355,1.000000
8438098,RE50356,1.000000


In [8]:
# Descriptive stats for Entropy (Measure 1)
entropy_stats = df['entropy_score'].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
entropy_stats['num_missing'] = df['entropy_score'].isna().sum()
entropy_stats['num_zeros'] = (df['entropy_score'] == 0).sum()

print("🔍 Descriptive Statistics for Entropy (Knowledge Novelty Index):")
print(entropy_stats)

# Descriptive stats for Technological Leap Score (Measure 2)
tech_leap_stats = data['tech_leap_score'].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
tech_leap_stats['num_missing'] = data['tech_leap_score'].isna().sum()
tech_leap_stats['num_zeros'] = (data['tech_leap_score'] == 0).sum()
tech_leap_stats['num_ones'] = (data['tech_leap_score'] == 1).sum()

print("\n🔍 Descriptive Statistics for Technological Leap Score:")
print(tech_leap_stats)

🔍 Descriptive Statistics for Entropy (Knowledge Novelty Index):
count          7.738361e+06
mean           2.853831e+00
std            1.220397e+00
min           -0.000000e+00
1%            -0.000000e+00
5%             6.931472e-01
25%            2.067481e+00
50%            2.865703e+00
75%            3.652313e+00
95%            4.886844e+00
99%            5.759233e+00
max            8.082816e+00
num_missing    0.000000e+00
num_zeros      1.908120e+05
Name: entropy_score, dtype: float64

🔍 Descriptive Statistics for Technological Leap Score:
count          8.085822e+06
mean           8.987757e-01
std            1.444546e-01
min            0.000000e+00
1%             2.000000e-01
5%             6.666667e-01
25%            8.666667e-01
50%            9.411765e-01
75%            1.000000e+00
95%            1.000000e+00
99%            1.000000e+00
max            1.000000e+00
num_missing    3.522780e+05
num_zeros      7.203900e+04
num_ones       2.166511e+06
Name: tech_leap_score, dtype: fl

In [11]:
# Remove NaNs just in case
df_clean = df.dropna(subset=['entropy_score'])

# Find the maximum entropy score
max_entropy = df_clean['entropy_score'].max()

# Get all patents with that max score
max_entropy_patents = df_clean[df_clean['entropy_score'] == max_entropy]
max_entropy_patents

,citing_patent_id,entropy_score
6845809,9535563,8.082816


In [13]:
pdf = pd.read_csv('g_patent_filtered.csv')

/var/folders/c6/b8__gkhn17s5n3tqscb4k5300000gn/T/ipykernel_33595/3097635988.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pdf = pd.read_csv('g_patent_filtered.csv')


In [14]:
result = pdf[pdf['patent_id'] == 9535563]
result

,patent_id,patent_type,patent_date,patent_title,wipo_kind
6301509,9535563,utility,2017-01-03,Internet appliance system and method,B2
